Assignment 10: Learn to Write Like Shakespeare
==============================================


Microsoft Forms Document: https://forms.office.com/r/xs1Xb1pe3g

In this assignment we will implement a simple recurrent network with one hidden layer.
We train this network on a medium-size poem "The Sonnet" written by William Shakespeare and use it for auto-completing sentences/phrases.

The data that we will use is originally provided here: http://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt

In [67]:
import os
import random
import torch

# download the data file
filename = "shakespeare.txt"
if not os.path.exists(filename):
  url = "http://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/"
  import urllib.request
  urllib.request.urlretrieve(url+filename, filename)
  print ("Downloaded datafile", filename)

# select to run everything on CUDA
device = torch.device("cuda")

We need to parse the data and turn it into a representation from which we can learn.
First, we need to count the number of unique characters to obtain the dimension $D$ of out input and output.
Then, we need to obtain one-hot encoding vectors for each of the characters.
Finally, we need to implement sequences and their according targets, using zero-padding where required.

Task 1: Data Characteristics
----------------------------

Load all text data from the file `shakespeare.txt`.
Count the number of unique characters contained in the poem. 
Here, we consider only lower-case characters to reduce the alphabet size.
At the same time, we also store the complete poem in a data variable.

Please make sure that you handle the newline character at the end of each line correctly and consistently.


In [68]:
# load all data from the text file
data = []
count = 0
with open(filename,"r") as f: ### data = ' '.join(f.read().lower().split())
  for line in f:
    if (line.strip()): ### 如果不是空行。 strip() 把换行符也删了；rstrip() 只删空格
      for c in line.lower().strip():
        data.append(c)
      data.append(" ")

# extract a list of all unique characters
characters = list(sorted(set(data)))

D = len(characters)
print (f"Collected a total of {len(data)} elements of {D} unique characters")

Collected a total of 93717 elements of 37 unique characters


Task 2: One-hot Encoding
------------------------

Each of the characters need to be represented by a one-hot encoding.
Create a dictionary that provides the encoding for each unique character.

In [69]:
one_hot = dict()
eye = torch.eye(D)
for i,c in enumerate(characters):
  one_hot[c] = eye[i]

Task 3: Sequence Coding
-----------------------

Write a function that provides the inputs and targets for a given sequence of the specified sequence length.
The last value of the target sequence should be the character of the given index.
If a character would be requested from outside of the data range, prepend the inputs (and the targets) with 0.
Assure that $\vec t^{\{s\}} = \vec x^{\{s+1\}}$ $\forall s<S$.

In [70]:
def sequence(index, S):
  # collect both input and target encodings
  inputs, targets = [], []
  # go through the sequence and turn characters into encodings
  zeros = torch.zeros(D)
  for i in range(index - S, index):
    inputs.append(one_hot[data[i]] if i >= 0 else zeros)
    targets.append(one_hot[data[i+1]] if i+1 >= 0 else zeros)
  
  return torch.stack(inputs), torch.stack(targets)

Test 1: Sequences
-----------------

Get a sequence for size 5 with index 2. Assure that the data and target vectors are as desired, i.e., the first elements are 0 vectors, and later one-hot encoded data is added.

In [71]:
# get sequence
x,t = sequence(index=2,S=5)
print("x:\n",x)
print("t:\n",t)
# perform checks
assert torch.all(x[:3] == 0)
assert torch.all(t[:2] == 0)
assert torch.all(torch.sum(x[3:],axis=1) == 1) ### torch.sum(x[3:],axis=1) => [1, 1] 所以要 torch.all( [1,1] ) == 1
assert torch.all(torch.sum(t[2:],axis=1) == 1)

x:
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]])
t:
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0

We use the standard data loader with a batch size of $B=256$. Theoretically, each training sample could have its own sequence length $S$. To enable batch processing, the sequence size must be the same for each element in the batch (otherwise it cannot be transformed as one large tensor). Thus, our dataset needs to have a fixed sequence size $S$.

我们使用标准的数据加载器，批量大小为𝐵=256。理论上，每个训练样本可以有自己的序列长度𝑆。为了能够进行批处理，批中的每个元素的序列大小必须相同（否则不能转化为一个大张量）。因此，我们的数据集需要有一个固定的序列尺寸𝑆。

Task 4: Dataset and Data Loader
-------------------------------
Implement a dataset that takes parameters $N$ (size of the dataset) and $S$ (size of the sequence).
In the `__getitem__` function, return the `sequence` (using the function of Task 3) for the sample with the given index, i.e., both the input and the target sequence.

实现一个数据集，它需要参数𝑁（数据集的大小）和𝑆（序列的大小）。在__getitem__函数中，返回具有给定索引的样本的序列（使用任务3的函数），即输入序列和目标序列都是如此。


In [72]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, data, S):
    self.S = S
    self.data = data ### self.data 就是 dataset当中的关键字
    self.N = len(self.data)

  def __getitem__(self, index):
    return sequence(index, self.S) ### x and t ∈ (S,D)

  def __len__(self):
    return self.N

B = 256
S = 20
dataset = Dataset(data=data, S=S)
data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=B, shuffle=True) # why shuffle True

Test 2: Data Sizes
------------------

Check that all samples in the dataset have the desired size and behavior.

In [73]:
# S = random.choice(range(1,20)) ### 不能加到 for loop 里面，会出错。
# dataset.S = S
for x,t in data_loader:
  # check that the data and targets are as expected
  assert x.shape[0] <= B and x.shape[1] == S and x.shape[2] == D
  assert t.shape[0] <= B and t.shape[1] == S and t.shape[2] == D
  assert torch.all(x[:,1:,:] == t[:,:-1,:]) ### (B,S,D) 

Task 5: Elman Network Implementation
------------------------------------

Manually implement an Elman network using one fully-connected layer for hidden, recurrent and output units.

Implement the processing of the input in the Elman network. Make sure that logit values are computed and returned for each element in the sequence. Try to use as much tensor processing as possible. Remember the shape of $X$ is $B\times S\times D$, and when going through the sequence, we need to process $\vec x^{\{s\}}$ separately, while working on all batch elements simultaneously.

使用一个全连接层的隐藏单元、循环单元和输出单元，手动实现一个埃尔曼网络。

在埃尔曼网络中实现对输入的处理。确保计算并返回序列中每个元素的logit值。尽量使用尽可能多的张量处理。记住𝑋的形状是𝐵×𝑆×𝐷，当通过序列时，我们需要分别处理𝑥⃗{𝑠}，同时对所有批处理元素进行处理。

In [74]:
class ElmanNetwork(torch.nn.Module):
  def __init__(self, D, K):
    super(ElmanNetwork,self).__init__()
    self.W1 = torch.nn.Linear(in_features=D, out_features=K)
    self.Wr = torch.nn.Linear(in_features=K, out_features=K)
    self.W2 = torch.nn.Linear(in_features=K, out_features=D)
    self.activation = torch.nn.PReLU()

  def forward(self, x):
    # get the shape of the data
    B, S, D = x.shape
    # initialize the hidden vector in the desired size with 0
    # remember to put it on the device
    h_s = torch.zeros(B, self.Wr.in_features, device=device)    ### 注意！！！h 有个 batch_size 在前面！！！
    # store all logits (we will need them in the loss function)
    Z = torch.empty(x.shape, device=device)
    # iterate over the sequence
    for s in range(S):
      # use current sequence item
      x_s = x[:,s,:]
      # print(x_s.shape) # (batch_size, D)
      # compute recurrent activation
      a_s = self.W1(x_s) + self.Wr(h_s)
      # print(a_s.shape) # (batch_size, K)
      # apply activation function
      h_s = self.activation(a_s) # (batch_size, K)
      # compute logit values
      z = self.W2(h_s) # (batch_size, K)
      # store logit value
      Z[:,s] = z ### Z ∈ (K, S)
      
    # return logits for all sequence elements
    return Z

Test 3: Network Output
----------------------

Instantiate an Elman network with arbitrary numbers for $D$ and $K$.
Generate training samples in a given format, forward them through the network and assure that the results are in the required dimensionality.

用任意的数字𝐷和𝐾实例化一个Elman网络。以给定的格式生成训练样本，通过网络转发，并保证结果符合要求的维度。

In [75]:
# instantiate test network
K = 1000
test_network = ElmanNetwork(D=D, K=K).to(device)

# create test input in size BxSxD
test_input = next(iter(data_loader))[0].to(device)
# get the network output
with torch.no_grad():
  test_output = test_network(test_input)
# check that the netowrk output size is as intended
print(test_input.shape)
print(test_output.shape)
assert test_output.shape == ( data_loader.batch_size, S, D ) # data_loader.batch_size 不等于 len(data_loader) 后者是 for loop 中的迭代次数，也就是 batch的个数 而不是size

torch.Size([256, 20, 37])
torch.Size([256, 20, 37])


To train the Elman network, we will use categorical cross-entropy loss, averaged over all samples in the sequence.
For each batch, we will use a different sequence size -- while the size inside a batch must stay the same.

According to the PyTorch documentation, the `CrossEntropyLoss` handles logits and targets in shape $B\times O\times\ldots$.
In our case, logits and targets are in dimension $B\times S\times O$.
Hence, we need to make sure that we re-order the indexes such that we fulfil the requirement; you might want to use the `permute` operator.

WARNING: `CrossEntropyLoss` will not complain when the order for the `CrossEntropyLoss` is wrong, just the results will be wrong.

为了训练Elman网络，我们将使用分类交叉熵损失，对序列中的所有样本取平均值。对于每个批次，我们将使用不同的序列大小--而一个批次内的大小必须保持不变。

根据PyTorch的文档，CrossEntropyLoss处理对数和目标的形状为𝐵×𝑂×...。在我们的例子中，对数和目标的维度是𝐵×𝑆×𝑂。因此，我们需要确保对索引进行重新排序，以便满足要求；你可能想使用permute操作符。

警告：当CrossEntropyLoss的顺序不对时，CrossEntropyLoss不会抱怨，只是结果会出错。


Task 6: Training Loop
---------------------
Instantiate the optimizer with an appropriate learning rate $\eta$ and the loss function.
Implement the training loop for 10 epochs -- more epochs will further improve the results.
Compute the average training loss per epoch.
Possibly, at the end of each batch, overwrite the `dataset.S` with a value randomly samples from $S\in[5,20]$.

Note that 10 epochs will train for about 2 minutes, if implemented in an optimized way, on the GPU. Non-optimized training will take considerably longer.

用适当的学习率𝜂和损失函数实例化优化器。执行10个epochs的训练循环 -- 更多的epochs将进一步改善结果。计算每个epoch的平均训练损失。可能的话，在每个批次结束时，用一个从𝑆∈[5,20]中随机抽取的值覆盖dataset.S。

请注意，如果以优化的方式实施，在GPU上，10个epochs将训练大约2分钟。非优化的训练将需要相当长的时间。

In [76]:
network = ElmanNetwork(D=D, K=K).to(device)
optimizer = torch.optim.Adam(
    params=network.parameters(),
    lr=1e-3
)
loss = torch.nn.CrossEntropyLoss()

for epoch in range(30):
  # create random sequence
  train_loss = 0.

  for x, t in data_loader:
    optimizer.zero_grad()
    # compute network output
    z = network(x.to(device))
    # compute loss, arrange order of logits and targets
    z = torch.permute(z, (0, 2, 1))
    t = torch.permute(t, (0, 2, 1)).to(device)
    J = loss(z,t)
    # compute gradient for this batch
    J.backward()
    # ------------------------------------------
    optimizer.step()   ###### 这个一定别忘了!!!!!!
    # ------------------------------------------
    # compute average loss
    train_loss += J.item() ### += J.item()
    # select a new sequence length S in [5,20]
    dataset.S = random.choice(range(5,21))

  # print average loss for training and validation
  print(f"\rEpoch {epoch+1}; train loss: {train_loss/len(data_loader):1.5f}")

Epoch 1; train loss: 2.15294
Epoch 2; train loss: 1.71587
Epoch 3; train loss: 1.52859
Epoch 4; train loss: 1.41165
Epoch 5; train loss: 1.32480
Epoch 6; train loss: 1.23157
Epoch 7; train loss: 1.16829
Epoch 8; train loss: 1.11256
Epoch 9; train loss: 1.08249
Epoch 10; train loss: 1.06097
Epoch 11; train loss: 1.03297
Epoch 12; train loss: 1.00243
Epoch 13; train loss: 0.99852
Epoch 14; train loss: 0.96766
Epoch 15; train loss: 0.98447
Epoch 16; train loss: 0.95497
Epoch 17; train loss: 0.98428
Epoch 18; train loss: 1.00163
Epoch 19; train loss: 0.96371
Epoch 20; train loss: 0.95873
Epoch 21; train loss: 0.93242
Epoch 22; train loss: 0.91882
Epoch 23; train loss: 0.95166
Epoch 24; train loss: 0.93091
Epoch 25; train loss: 0.94757
Epoch 26; train loss: 0.89890
Epoch 27; train loss: 0.91782
Epoch 28; train loss: 0.93805
Epoch 29; train loss: 0.96914
Epoch 30; train loss: 0.93104


Task 7: Text Encoding
---------------------
For a given text (a sequence of $S$ characters), provide the encoding $\mathcal X \in R^{B\times S\times D}$.
Assure that the batch index $B=1$ is added to the encoding, so that the network is able to handle it.

对于一个给定的文本（一串𝑆字符），提供编码 $\mathcal X \in R^{B\times S\times D}$。确保在编码中加入批索引𝐵=1，以便网络能够处理它。

In [109]:
def encode(text):
  # S = len(text)
  # x,_ = sequence(index=S,S=S) ### 不能直接用 sequence 啊, 前面写的 sequence 用了一个全局变量 data
  encoding = torch.stack([one_hot[c] for c in text]).unsqueeze(dim=0)
  return encoding

print(encode("abc").shape)

torch.Size([1, 3, 37])


Task 8: Next Element Prediction
-------------------------------

Implement a function that return the next character from the logits returned by the network.
Note that the logits are in dimension $\mathcal Y \in \mathbb R^{B\times S\times D}$ with $B=1$, and we are generally only interested in the prediction for the last sequence item.

实现一个函数，从网络返回的logits中返回下一个字符。注意，logits的维度为$\mathcal Y \in \mathbb R^{B\times S\times D}$，其中𝐵=1，我们一般只对最后一个序列项的预测感兴趣。

Select the character with the highest SoftMax probability $\max_o z^{\{S\}}_o$ and append this character to the `text`.
Alternatively, we can also randomly draw a character based on the SoftMax probability distribution $\vec y^{\{S\}}$. `random.choices` provides the possibility to pass a list of characters and a list of probabilities.

选择具有最高SoftMax概率的字符$\max_o z^{\{S\}}_o$，并将此字符追加到文本中。
另外，我们也可以根据SoftMax概率分布 $\vec y^{\{S\}}$. 随机抽取一个字符。`random.choice`提供了传递一个字符列表和一个概率列表的可能性。

In [110]:
def predict(z, use_best): ### z.shape = (B,S,D) 在这里 B = 1
  # select the appropriate logits
  z_S = z[0][-1] ### B=1 所以[0] 取到(S,D); 因为只对最后一个 logits 感兴趣，所以取到[D], s 是最后一个 character
  if use_best:
    # take character with maximum probability
    max_idx = torch.argmax(z_S)
    next_char = characters[max_idx]
  else:
    # sample character based on class probabilities
        ###### 如果不写 dim=0 会怎么样？ 会报错； 这样写也可以 y_S = torch.nn.Softmax(dim=0)(z_S)
        ###### 如果不写.cpu() 会怎么样？ 也不会怎么样, 最好还是写上吧
    y_S = torch.softmax(z_S, dim=0).cpu()
    next_char = random.choices(characters, weights=y_S)[0] ####### 这里要按权重抽样，必须写 random.choices(xxx, weights=) 别忘了【s】!!! 其返回的是一个list！！！
  return next_char

Task 9: Sequence Completion
---------------------------

Write a function that takes a `seed` text which it will complete with the given number of characters.
Write a loop that turns the current `text` into an encoded sequence of its characters using the function from Task 7.
Forward the text through the network and take the prediction of the last sequence item $\vec z^{\{S\}}$ using the function from Task 8.
Append this to the current text (remember that Python strings are immutable).
Repeat this loop 80 times, and return the resulting `text`.

编写一个函数，接收一个种子文本，它将用给定的字符数完成。编写一个循环，使用任务7中的函数将当前文本变成其字符的编码序列。通过网络转发文本，并使用任务8中的函数对最后一个序列项𝑧⃗ {𝑆}进行预测。将其追加到当前文本中（记住，Python字符串是不可改变的）。重复这个循环80次，并返回结果文本。

In [111]:
def sequence_completion(seed, count, use_best):
  # we start with the given seed
  text = seed
  print(seed)
  for i in range(count):
    # turn current text to one-hot batch
    x = encode(text) ### 这里要把持续增加序列的 text encode 成为 one_hot 序列 (B,S,D); B = 1,S++
    z = network(x.to(device)).cpu()
    # predict the next character
    next_char = predict(z, use_best)
    # append character to text
    text += next_char
    
  return text

Task 10: Text Production
-----------------------

Select several seeds (such as `"the ", "beau", "mothe", "bloo"`) and let the network predict the following 80 most probable characters, or using probability sampling.
Write the completed sentences to console.

In [113]:
seeds = ["the ", "beau", "mothe", "bloo"]

for seed in seeds:
  best = sequence_completion(seed, 80, True)
  # print seed and text
  print (f"\"{seed}\" -> \"{best}\"")
  sampled = sequence_completion(seed, 80, False)
  # print seed and text
  print (f"\"{seed}\" -> \"{sampled}\"")
  print()

the 
"the " -> "the stars in secr true to his sweet up-locked treasure, now count not all the better"
the 
"the " -> "the othes that i before have what should it as the part, then love be taken to hell "

beau
"beau" -> "beauty's summer dead. let not my love be called idole mounning doas disgrace. theref"
beau
"beau" -> "beauty hath a face hate the imbubutation i not fleese mighod, so thou preceither we "

mothe
"mothe" -> "mother's child, though not true, and thine eyes my knowledge thee, where be nothing n"
mothe
"mothe" -> "mother's child of mine can say more that like of hearta thould example where you were"

bloo
"bloo" -> "blood warm when thou well thou art, within thine own desert, as thou be not times in"
bloo
"bloo" -> "blood, that it could soy, but he powell min on juck permed it in thy part, to leave "

